## Inicio - importação das bibliotecas

In [1]:
import pandas as pd
import re

### Abrindo base de dados

In [2]:
colunas = ['cd_causa', 'cd_atendimento', 'ds_Acao_Judicial', 'ds_fatos', 'ds_Pedidos', 'ds_Qualificacao']

# LINHA DE SELECAO DO INPUT
df = pd.read_csv(
    "dataset_clinica20252.csv",
    sep="|",               
    encoding="utf-8"     
)

print(df.shape)
print(df.head(20))

(19800, 6)
      cd_causa             cd_atendimento                 ds_Acao_Judicial  \
0   CIB0500064  0825789-84.2025.8.18.0140  90 - ACAO DE REPARACAO DE DANOS   
1   CIB0505587  1004697-72.2025.8.26.0066  90 - ACAO DE REPARACAO DE DANOS   
2   CIB0508201  0800423-07.2025.8.15.0761  90 - ACAO DE REPARACAO DE DANOS   
3   CIB0514647  1004875-69.2025.8.26.0438  90 - ACAO DE REPARACAO DE DANOS   
4   CIB0500604  0010630-50.2025.8.27.2706  90 - ACAO DE REPARACAO DE DANOS   
5   CIB0511068  0092601-36.2025.8.05.0001  90 - ACAO DE REPARACAO DE DANOS   
6   CIB0515030  0801575-51.2025.8.18.0068  90 - ACAO DE REPARACAO DE DANOS   
7   CIB0515018  0801452-38.2025.8.18.0073  90 - ACAO DE REPARACAO DE DANOS   
8   CIB0515702  0800895-81.2025.8.10.0038  90 - ACAO DE REPARACAO DE DANOS   
9   CIB0514229  0148716-17.2025.8.04.1000  90 - ACAO DE REPARACAO DE DANOS   
10  CIB0514230  0148734-38.2025.8.04.1000  90 - ACAO DE REPARACAO DE DANOS   
11  CIB0524760  0161843-22.2025.8.04.1000  90 - ACAO 

### Etapa 1 - Função de captar valor pedido e número do CNPJ

In [10]:
def cria_colunas(df: pd.DataFrame):
    MAPA_TJ = {
        "01":"TJAC","02":"TJAL","03":"TJAP","04":"TJAM","05":"TJBA",
        "06":"TJCE","07":"TJDF","08":"TJES","09":"TJGO","10":"TJMA",
        "11":"TJMT","12":"TJMS","13":"TJMG","14":"TJPA","15":"TJPB",
        "16":"TJPR","17":"TJPE","18":"TJPI","19":"TJRJ","20":"TJRN",
        "21":"TJRS","22":"TJRO","23":"TJRR","24":"TJSC","25":"TJSE",
        "26":"TJSP","27":"TJTO"
    }
    PADRAO_CNJ = re.compile(r"(\d{7})(\d{2})(\d{4})(\d)(\d{2})(\d{4})")
    SM = 1518 * 40

    def process_row(r):
        # CNPJ
        pad_cnpj = [
            r'(\d{2}[.,]\d{3}[.,]\d{3}[/\\]\d{4}[-\s]?\d{2})',
            r'(\d{2}[.,]\d{3}[.,]\d{3}[.,]\d{4}[-\s]?\d{2})',
            r'(\d{2}\s*[.,]\s*\d{3}\s*[.,]\s*\d{3}\s*[/\\]\s*\d{4}\s*[-\s]?\s*\d{2})',
            r'(\d{2}[\s.,]*\d{3}[\s.,]*\d{3}[\s.,/\\]*\d{4}[\s\-]*\d{2})',
            r'(?:CNPJ|cnpj)[\s:]*(?:n[ºo°]?\.?[\s:]*)(\d{2}[.,\s]*\d{3}[.,\s]*\d{3}[/\\.,\s]*\d{4}[-\s]*\d{2})',
            r'\b(\d{14})\b'
        ]
        cnpjs = set()
        for rx in pad_cnpj:
            for m in re.findall(rx, str(r['ds_Qualificacao']), re.IGNORECASE):
                d = re.sub(r'\D','',m)
                if len(d)==14:
                    cnpjs.add(d)
        cnpj = ",".join(sorted(cnpjs)) or ""

        # Valor causa
        s = str(r['ds_Pedidos'])
        geral = re.findall(r'R\$\s*([0-9]+(?:\.[0-9]{3})*,[0-9]{2}|[0-9]+,[0-9]{2})', s)
        pad_valor = [
            r'(?:dar-se-á|dá-se|dão)\s+(?:à|a)\s+causa.*?R\$\s*([0-9]+(?:\.[0-9]{3})*,[0-9]{2}|[0-9]+,[0-9]{2})',
            r'valor\s+da\s+causa[:\s]+R\$\s*([0-9]+(?:\.[0-9]{3})*,[0-9]{2}|[0-9]+,[0-9]{2})'
        ]
        valor = 0.0; conf = "baixa"
        for rx in pad_valor:
            m = re.search(rx, s, re.IGNORECASE)
            if m:
                vs = m.group(1)
                valor = float(vs.replace('.','').replace(',','.'))
                conf = "alta"
                break
        else:
            if geral:
                vs = geral[-1]
                valor = float(vs.replace('.','').replace(',','.'))
                conf = "baixa"

        # dt distribuição
        t = str(r['ds_fatos'])
        m = re.search(r'Assinado\s+eletronicamente\s+por:.*?-\s*(\d{2}/\d{2}/\d{4})', t, re.IGNORECASE|re.DOTALL)
        if m:
            d,mo,y = m.group(1).split('/')
            dt = f"{y}-{mo}-{d}"
        else:
            dts = re.findall(r'\b([0-3]\d)[/.\-]([01]\d)[/.\-]((?:19|20)\d{2})\b', t)
            if dts:
                d,mo,y = dts[-1]
                dt = f"{y}-{mo}-{d}"
            else:
                dt = ""

        # UF
        s2 = re.sub(r'\D','', str(r['cd_atendimento']))
        m2 = PADRAO_CNJ.fullmatch(s2)
        if m2 and m2.group(4)=="8":
            tr = m2.group(5)
            tj = MAPA_TJ.get(tr)
            uf = tj[-2:] if tj else ""
        else:
            uf = ""

        # tipo vara
        tipo = "JE" if valor < SM else "G1"

        return pd.Series({
            'cd_atendimento': r['cd_atendimento'],
            'cnpj': cnpj,
            'valor_causa': valor,
            'dt_distribuicao': dt,
            'tipo_vara': tipo,
            'UF': uf
        })

    resultado = df.apply(process_row, axis=1)
    resultado.to_excel('output.xlsx', index=False)

    return


In [11]:
cria_colunas(df)

### Etapa 2 - Função de filtrar processos de empréstimo consignado